This file contains the baseline model and the final model I built on F.G.'s handwritten journal image cropped tiles. The main file for the project. Also, in this file I select the most frequent 7 classes and the model is built to predict these classes. I used the top 7 most frequent since they have at least 300 labeled examples.



First I list all the imports used this file.

In [0]:
import os
import numpy as np
import keras
from keras.models import model_from_json
from skimage.transform import resize
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from random import seed, sample
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from pprint import pprint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

ld.tar contains the tarred labeled_data directory from https://github.com/Vinaymy/wtf/tree/master/data/labeled_data. The next line untars this file and collects data in the local directory of execution.

In [0]:
!tar -xvf ld.tar

Next I load up the cropped labeled images into memory and map them to class labels. I map upper and lower case English alphabet to the same class. Along with 10 digits this gives me 36 classes. The labels for the classes are clear from the mapping dictionary in the code below.

Also, I set a frequency lower bound of 300 for selecting a character class for the prototype CNN, so that I have enough data to feed the CNN. 7 character classes pass this filter - A/a, D/d, E/e, N/n, O/o, R/r and S/s.

Also, I limit the number of examples used for each of the classes to exactly 300 even though some classes (e.g. E/e) have more examples in the labeled set. This is so I have a balanced distribution of all classes. I empirically found that imbalanced classes cause the model to overfit and do terribly on the test set.

In [32]:
rootDir = './labeled_data'
n_images = 10000
images = np.zeros((n_images,) + (20, 20))
minimum_number_of_examples = 300 # I came up with this after inspecting the distribution of frequencies of the labeled classes. 
# I decided that images with fewer than examples is too few to try predict.

labels = []
lab = {
    '0': 1, '1': 2, '2': 3, '3':4, '4':5, '5':6, '6':7, '7':8, '8':9, '9':10,
    'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 
    'N': 24, 'O': 25, 'P': 26, 'Q': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'X': 34, 'Y': 35, 'Z': 36,
    'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 
    'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36
}
j = 0
ct = {}
for dirName, subdirList, fileList in os.walk(rootDir):
    i = 0
    for fname in fileList:
        if 'png' not in fname or fname[:3] !='IMG':
            continue
        full_path = os.path.join(dirName, fname)
        
        if dirName[-1] in ['E', 'e', 'A', 'a', 'T', 't', 'S', 's', 'D', 'd', 'N', 'n', 'O', 'o'] \
        and dirName[-2] == '/' and ct.get(dirName[-1].lower(), 0) < minimum_number_of_examples:
            ct[dirName[-1].lower()] = ct.get(dirName[-1].lower(), 0) + 1
        else:
            continue
        
        # Resize image  to 20 X 20
        images[j] = resize(imread(full_path, as_grey=True), (20, 20)) 
        labels.append(lab[dirName[-1]])
        j += 1
        i += 1
images = images[:len(labels)]

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In the next part I inspect the number of examples selected for each label and ensure that it is 300 exactly.


In [0]:
count = Counter(labels)

In [34]:
count.most_common() # 7 labels of and balanced since I picked 300 of each

[(29, 300), (15, 300), (14, 300), (30, 300), (24, 300), (25, 300), (11, 300)]

Let us examine the labels that passed the filter of at least 300 examples.

In [67]:
np.unique(labels) 

array([11, 14, 15, 24, 25, 29, 30])

The above number labels correspond to [A/a, D/d. E/e, N/n, O/o, R/r, S/s]

Perform 80% - 20% split of examples to get a traiing-set/testing set split. Then we'll have 1680 training set image tiles and 420 test set image tiles.

In [61]:
k = sample(range(len(images)), len(images))
im_shuf = images[k]
labels_shuf = np.array(labels)[k]
x_train, x_test, y_train, y_test = train_test_split(im_shuf, labels_shuf, random_state=2, train_size=0.8)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Examine the size of the training set and confirm its 1680 examples.

In [62]:
y_train.shape

(1680,)

Build a baseline model and record its accuracy on the test set for reference. I build a linear kernel SVM with grid search on hyperparameter C. I get an accuracy of 0.47 on the training set and 0.23 on the test set for the baseline model. 

In [73]:
base_model = LinearSVC()
param_grid = {'C':  list(np.arange(0.1,1.5,0.1))}

gs = GridSearchCV(base_model, param_grid, n_jobs=-1, cv=3, verbose=4)
x_train_resh = x_train.reshape((x_train.shape[0], -1))
x_test_resh = x_test.reshape((x_test.shape[0], -1))
gs.fit(x_train_resh, y_train)
pprint(sorted(gs.grid_scores_, key=lambda x: -x.mean_validation_score))

y_pred = gs.predict(x_test_resh)
print ('Test set shape: ', x_test_resh.shape)
print ('Target shape: ', y_test.shape)
print ('Accuracy on train set: ', accuracy_score(y_train, gs.predict(x_train_resh)))
print ('Accuracy on test set: ', accuracy_score(y_test, y_pred))

Fitting 3 folds for each of 14 candidates, totalling 42 fits
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] .................................. C=0.1, score=0.225577 -   6.3s
[CV] C=0.1 ...........................................................
[CV] .................................. C=0.1, score=0.210714 -   7.0s
[CV] C=0.2 ...........................................................
[CV] .................................. C=0.1, score=0.222621 -   8.0s
[CV] C=0.2 ...........................................................
[CV] .................................. C=0.2, score=0.211368 -  10.2s
[CV] C=0.2 ...........................................................
[CV] .................................. C=0.2, score=0.207143 -  10.2s
[CV] C=0.30000000000000004 ...........................................
[CV] .................................. C=0.2, score=0.211849 -  10.4s
[CV] C=0.3000000

[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.7min


[CV] .................................. C=0.8, score=0.206039 -   9.9s
[CV] C=0.8 ...........................................................
[CV] .................................. C=0.8, score=0.196429 -   9.7s
[CV] C=0.9 ...........................................................
[CV] .................................. C=0.8, score=0.197487 -   9.8s
[CV] C=0.9 ...........................................................
[CV] .................................. C=0.9, score=0.204263 -   9.9s
[CV] C=0.9 ...........................................................
[CV] .................................. C=0.9, score=0.192857 -  10.0s
[CV] C=1.0 ...........................................................
[CV] .................................. C=0.9, score=0.201077 -   9.9s
[CV] C=1.0 ...........................................................
[CV] .................................. C=1.0, score=0.211368 -   9.8s
[CV] C=1.0 ...........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:  3.4min finished


[mean: 0.21964, std: 0.00642, params: {'C': 0.1},
 mean: 0.21012, std: 0.00211, params: {'C': 0.2},
 mean: 0.20655, std: 0.00537, params: {'C': 0.30000000000000004},
 mean: 0.20536, std: 0.00428, params: {'C': 0.4},
 mean: 0.20417, std: 0.00337, params: {'C': 1.1},
 mean: 0.20357, std: 0.00554, params: {'C': 1.0},
 mean: 0.20119, std: 0.00247, params: {'C': 0.5},
 mean: 0.20000, std: 0.00508, params: {'C': 0.6},
 mean: 0.20000, std: 0.00430, params: {'C': 0.8},
 mean: 0.19940, std: 0.00481, params: {'C': 0.9},
 mean: 0.19643, std: 0.00614, params: {'C': 0.7000000000000001},
 mean: 0.19345, std: 0.00641, params: {'C': 1.2000000000000002},
 mean: 0.19345, std: 0.00627, params: {'C': 1.3000000000000003},
 mean: 0.19286, std: 0.01577, params: {'C': 1.4000000000000001}]
Test set shape:  (420, 400)
Target shape:  (420,)
Accuracy on train set:  0.4720238095238095
Accuracy on test set:  0.23333333333333334


Next I build the target model CNN. I use a batch size of 128 for training, this is a pretty good batch size since it's reasonable large compared to the total data size, and also runs pretty efficiently. Also, I one hot encode the target labels.

In [0]:
num_classes = len(uniques)+1
batch_size = 128

X_train = x_train.reshape(x_train.shape[0], 20, 20 , 1).astype('float32')
X_test = x_test.reshape(x_test.shape[0], 20, 20 , 1).astype('float32')

uniques, ids_tr = np.unique(y_train, return_inverse=True)
uniques, ids_te = np.unique(y_test, return_inverse=True)

Y_train = to_categorical(ids_tr, num_classes)
Y_test = to_categorical(ids_te, num_classes)

Examine a sample from the one hot encoded test set. The below is an example of an e. Index 0 in the one-hot encoding is not used.

In [66]:
Y_test[0]

array([0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

Specify the model architecture and train it and test it. I get a training accuracy of 0.941 and a test accuracy of 0.807. 

I used 70 epochs and batch size of 128 arrived through trial and error. 

I add a wrapper method for creating the model, since in the next steps I perform a 5-fold cross validation analysis and I want to modularize the creation of the model architecture so that I can reuse this code.

In [0]:
def create_network(num_classes):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(20, 20, 1)))

    model.add(Conv2D(64, (5, 5), activation='relu'))

    model.add(Conv2D(128, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    return model

Next call the above method and actually create a model and train it on the training set and test it on the test set.

In [0]:
model = create_network(num_classes)
model.summary()
model.fit(X_train, Y_train, batch_size=batch_size,
                        nb_epoch=70,
                        verbose=1)

In [42]:
scores = model.evaluate(X_test, Y_test, verbose = 10 )
print ( scores )


[1.2459780988239106, 0.6928571428571428]


Next I generate a multi-class confusion matrix. 

In [0]:
y_predict_non_category = [ np.unique(labels)[np.argmax(t)] for t in y_predict ]

conf_mat = confusion_matrix(y_test, y_predict_non_category)

In [73]:
conf_mat

array([[54,  0,  0,  2,  2,  0,  0],
       [ 8, 56,  0,  0,  1,  1,  1],
       [ 1,  0, 44,  1,  1,  2,  0],
       [ 0,  0,  0, 60,  0,  0,  0],
       [ 1,  0,  0,  5, 53,  1,  0],
       [ 3,  0,  0,  2,  1, 65,  1],
       [ 1,  0,  0,  0,  0,  4, 49]])

Next, let us do some robustness analysis and check for overfitting through a 5 fold cross validation across the full dataset, to understand the training process and performance better. I print out the training accuracy and loss at every epoch and also testing loss and accuracy at the end training for each fold. 

This analusis shows that the model is training quite well and showing a healthy change in loss and accuracy over 70 epochs consistently across the folds. Hoeever, I do see some signs of overfitting, since the accuracy on the holdout set is typically in the 70 - 80 % range. This indicates reasonably clearly that the very next step should be to get more labeled data. However due to class imbalance I could be selective about what type of labeled data to collect. I have more details on this in my report. 

Another strategy I may need to try after this project is to experiment with additional dropout.

In [30]:
kf = KFold(n_splits=5)

X_valid = im_shuf.reshape(im_shuf.shape[0], 20, 20 , 1).astype('float32')

uniques, ids_valid = np.unique(labels_shuf, return_inverse=True)

Y_valid = to_categorical(ids_valid, num_classes)

for train_indices, test_indices in kf.split(X_valid):
    model = create_network(num_classes = len(uniques)+1)
    model.fit(X_valid[train_indices], Y_valid[train_indices], batch_size=batch_size,
                        nb_epoch=70,
                        verbose=1)
    scores = model.evaluate(X_valid[test_indices], Y_valid[test_indices], verbose = 10 )
    print(scores)

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/70
1680/1680 [==============================] - 7s 4ms/step - loss: 2.0431 - acc: 0.1381
Epoch 2/70
1680/1680 [==============================] - 6s 4ms/step - loss: 2.0057 - acc: 0.1381
Epoch 3/70
1680/1680 [==============================] - 6s 4ms/step - loss: 2.0037 - acc: 0.1464
Epoch 4/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9839 - acc: 0.1357
Epoch 5/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9723 - acc: 0.1494
Epoch 6/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9845 - acc: 0.1369
Epoch 7/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9684 - acc: 0.1601
Epoch 8/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9740 - acc: 0.1435
Epoch 9/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9698 - acc: 0.1464
Epoch 10/70
1680/1680 [==============================] - 6s 4ms/step - loss: 1.9670 - acc: 0.1607
Epoch 11/70
1680/1680 [======